## **1. 데이터 준비하기(Custom Data 구축)**
- Roboflow에서 제공하는 Training Dataset을 이용

In [3]:
from sklearn.model_selection import train_test_split
import os
import cv2

In [4]:
from roboflow import Roboflow
rf = Roboflow(api_key="RWjOZsvvw91zodI8DjZb")
project = rf.workspace("230816newcrackwallpeelingrebar").project("opendataset_mix")
version = project.version(3)
dataset = version.download("yolov8")

rf2 = Roboflow(api_key="RWjOZsvvw91zodI8DjZb")
project2 = rf.workspace("sri-ramakrishna-engineering-college-qdkxv").project("cracks-oa1tt")
version2 = project.version(1)
dataset2 = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.26, to fix: `pip install ultralytics==8.0.196`
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.26, to fix: `pip install ultralytics==8.0.196`


In [5]:
print(type(dataset))
print(type(dataset2))

# 데이터셋이 다운로드된 경로 확인
dataset_path = dataset.location
print(f"Dataset downloaded to: {dataset_path}")

<class 'roboflow.core.dataset.Dataset'>
<class 'roboflow.core.dataset.Dataset'>
Dataset downloaded to: /home/ubuntu/Downloads/opendataset_mix-3


In [6]:
!pwd

/home/ubuntu/Downloads


## **2) yaml 파일 설정** (데이터셋 위치 알려주는 config file)
- YAML : 데이터 표현 양식의 한 종류
- 기본적으로 들여쓰기(indent)를 원칙으로하며 데이터는 Map(key-value)형식으로 작성

### 2-1) roboflow 에서 제공되는 data.yaml 파일 확인

In [7]:
# yaml 파일을 학습이 가능하도록 경로 설정.
# key-value 데이터인 dict 데이터타입으로 data['train'], data['val'], data['nc'], data['names'] 에 넣어주는데,
# 가장 중요한 부분은 데이터 경로 설정임.

import yaml
# 디렉토리 정보, 클래스 이름(names), 클래스 수(nc) 지정하기
import yaml
with open('/home/ubuntu/Downloads/opendataset_mix-3/data.yaml') as f:
    data = yaml.full_load(f)

print(data)
data['train'] = '/home/ubuntu/Downloads/opendataset_mix-3/train/'
data['test'] = '/home/ubuntu/Downloads/opendataset_mix-3/test/'
data['val'] = '/home/ubuntu/Downloads/opendataset_mix-3//valid/'

# 데이터 경로와 클래스 정보를 저장하고 있는 딕셔너리 객체 data를 YOLOv8 학습에 필요한 새로운 이름으로 저장
with open('/home/ubuntu/Downloads/opendataset_mix-3/data.yaml', 'w') as f:
    yaml.dump(data, f)

# Aquarium_Data.yaml 읽어서 화면에 출력
with open('/home/ubuntu/Downloads/opendataset_mix-3/data.yaml', 'r') as f:
  aquarium_yaml = yaml.safe_load(f)
  display(aquarium_yaml)

{'names': ['crack'], 'nc': 1, 'roboflow': {'license': 'CC BY 4.0', 'project': 'opendataset_mix', 'url': 'https://universe.roboflow.com/230816newcrackwallpeelingrebar/opendataset_mix/dataset/3', 'version': 3, 'workspace': '230816newcrackwallpeelingrebar'}, 'test': '/home/ubuntu/Downloads/opendataset_mix-3/test/', 'train': '/home/ubuntu/Downloads/opendataset_mix-3/train/', 'val': '/home/ubuntu/Downloads/opendataset_mix-3//valid/'}


{'names': ['crack'],
 'nc': 1,
 'roboflow': {'license': 'CC BY 4.0',
  'project': 'opendataset_mix',
  'url': 'https://universe.roboflow.com/230816newcrackwallpeelingrebar/opendataset_mix/dataset/3',
  'version': 3,
  'workspace': '230816newcrackwallpeelingrebar'},
 'test': '/home/ubuntu/Downloads/opendataset_mix-3/test/',
 'train': '/home/ubuntu/Downloads/opendataset_mix-3/train/',
 'val': '/home/ubuntu/Downloads/opendataset_mix-3//valid/'}

## **3.모델 객체 선언하고 학습하기**

### 1) 모델 객체 선언하기

In [8]:
from ultralytics import YOLO
model = YOLO('yolov8n-seg.pt')  #  model = YOLO('yolov8n-seg.pt')

# 모델 학습
model.train(data="/home/ubuntu/Downloads/opendataset_mix-3/data.yaml", epochs=100)

# 학습 완료된 모델 저장
model.save('yolov8-seg-trained.pt')
print("Model saved to yolov8-seg-trained.pt")

New https://pypi.org/project/ultralytics/8.2.31 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.26 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060, 7918MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/home/ubuntu/Downloads/opendataset_mix-3/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retin

Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
train: Scanning /home/ubuntu/Downloads/opendataset_mix-3/train/labels.cache... 1
val: Scanning /home/ubuntu/Downloads/opendataset_mix-3/valid/labels.cache... 16 


Plotting labels to runs/segment/train10/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/segment/train10
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/11 [00:00<?, ?it/s]Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
      1/100      2.82G      1.406      4.226      2.768      1.446         54   
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all         16         19    0.00396          1      0.333      0.136     0.0025      0.632     0.0257    0.00449


...

    100/100      2.59G     0.4621     0.8261     0.3623     0.9097         39   
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all         16         19      0.814      0.947      0.867      0.618      0.859          1       0.94      0.481



100 epochs completed in 0.053 hours.
Optimizer stripped from runs/segment/train10/weights/last.pt, 6.8MB
Optimizer stripped from runs/segment/train10/weights/best.pt, 6.8MB

Validating runs/segment/train10/weights/best.pt...
Ultralytics YOLOv8.2.26 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060, 7918MiB)
YOLOv8n-seg summary (fused): 195 layers, 3258259 parameters, 0 gradients, 12.0 GFLOPs


Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
                 Class     Images  Instances      Box(P          R      mAP50  m


                   all         16         19      0.849      0.891      0.865      0.652        0.9      0.943      0.948      0.505
Speed: 0.2ms preprocess, 1.7ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/segment/train10
Model saved to yolov8-seg-trained.pt


In [9]:
# PyTorch window 에 설치
#- https://pytorch.org/get-started/locally/
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# 위 실행해서 안되면 아래 명령 실행
#!conda install pytorch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 pytorch-cuda=12.1 -c pytorch -c nvidia

In [13]:
# 커스텀 데이터로 학습하였기 때문에 클래수 수의 변경됨을 확인할 수 있음
print(type(model.names), len(model.names))

print(model.names)

<class 'dict'> 80
{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell p

### ▲ train 과정중에 loss, accuracy 등의 graph 데이터는 runs/detect/train/ 에 있는 results.csv 와 results.png 통해서 확인가능함

### 3) 테스트 이미지 데이터 생성 및 확인

In [14]:
# 테스트 이미지
from glob import glob

test_image_list = glob('/content/TACO_Data/test/images/*')
test_image_list.sort()

for i in range(len(test_image_list)):
    print('i = ',i, test_image_list[i])

## **7.학습된 모델 내보내기**

In [4]:
# 내보낸 모델을 로드하여 사용
model = YOLO('crack_trace.pt')

NameError: name 'YOLO' is not defined

## 8. 테스트 이미지로 테스트 또는 웹캠 이용해 실시간 추론하기

In [ ]:
# 학습된 모델을 로드합니다.
model = YOLO('crack_trace.pt')

# 웹캠 초기화
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 모델을 사용하여 프레임에서 객체 감지
    results = model(frame)

    # 결과를 프레임에 그리기
    annotated_frame = results.render()[0]

    # 화면에 표시
    cv2.imshow("YOLOv8 Real-Time Detection", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()